In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import time
import os

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
path = r'D:\РЭШ\Research\PostThesis\data'
path_bankdata = path+r'\bankdata'

# 1. Обработка данных в переменные

Данные отсутствуют за 2021-12, ..., 2023-03.<br><br>
Так как форма менялась в июне 2021 и в марте 2019, то создаём 3 функции: <br> 
    1) до марта 2019 ```df_var_to201812```, <br>
    2) с марта 2019 до июня 2021 ```df_var_201903```, <br> 
    3) с июня 2021 ```df_var_202106```.

In [4]:
def yearmonth_to_quarter(df, col_year, col_month):
    """Year and month numbers to format YYYYqN (2003, 09 to 2003q3)"""
    month_to_quarter = {3: 'q1', 6: 'q2', 9: 'q3', 12: 'q4'}
    quarters = df[col_year].astype(str) + df[col_month].replace(month_to_quarter)
    return quarters

def quarter_to_date(a):
    """Quarter of the format 2003q1 to date format 2003-01-01, a is an array"""
    quarter_to_month = {'1': '01', '2': '04', '3': '07', '4': '10'}
    dates = []
    for q in a:
        dates.append(datetime.strptime(q[:4] + '-' + quarter_to_month[q[-1]] + '-01', '%Y-%m-%d'))
    return dates

In [5]:
def df_var_202106(df):
    """From the central bank data, extract variables for the work. From 2021-06 until ..."""
    # 34 Всего источников собственных средств группы и малых акционеров (участников)
    
    df = df.dropna(subset=['14 Всего активов'])
    
    df['quarter'] = yearmonth_to_quarter(df, 'dt_year', 'dt_month')

    df['Name'] = df['Полное или сокращенное фирменное наименование головной кредитной организации']
    df['Total_assets'] = df['14 Всего активов']
    df['Net_worth'] = df['34 Всего источников собственных средств группы и малых акционеров (участников)'] #!
    df['Net_income'] = df['27 Прибыль (убыток) за отчетный период, в том числе:']
    
    df['Loans'] = (df['4.1 кредиты (займы) и дебиторская задолженность, в том числе:'] +
                   df['5.3 кредиты (займы)'] + df['6.3 кредиты (займы)']) 
    df['Admin_expenses'] = df['21 Административные и прочие операционные расходы, в том числе:']
    df['Rel_Admin_expenses'] = df['Admin_expenses'] / df['Total_assets']
    #Total holdings of securities
    df['Securities'] = (df['4.2 долговые ценные бумаги'] + df['5.1 долговые ценные бумаги, в том числе:'] +
                        df['5.2 долевые ценные бумаги'] + df['6.1 долговые ценные бумаги, в том числе:'] +
                        df['6.2 долевые ценные бумаги, в том числе:'])
    # non-interest expenses
    df['NIE'] = (df['14 Комиссионные расходы'] + 
                 df['21 Административные и прочие операционные расходы, в том числе:'])
    # non-interest income
    df['NII'] = (df['12 Доходы в виде дивидендов'] + df['13 Комиссионные доходы'] + df['19 Прочие операционные доходы'])
    # interest income on loans
    df['II_Loans'] = (df['1.2.1 от кредитов (займов), предоставленных кредитным организациям, и дебиторской задолженности'] +
                      df['1.2.2 от кредитов (займов), предоставленных клиентам, не являющимся кредитными организациями, и дебиторской задолженности'])
    #interest expenses on deposits
    df['IE_Deposits'] = (df['2.1 по привлеченным средствам кредитных организаций'] +
                         df['2.2 по привлеченным средствам клиентов, не являющихся кредитными организациями'])
    df['Leverage'] = df['Net_worth'] / df['Total_assets']
    df['Deposits'] = (df['15 Финансовые обязательства, оцениваемые по амортизированной стоимости, в том числе:'] + 
                      df['16 Финансовые обязательства, оцениваемые по справедливой стоимости через прибыль или убыток, в том числе:'])
    df['Pure_deposits'] = (df['15.4 средства клиентов, не являющихся кредитными организациями'] +
                           df['15.5 средства физических лиц'] +
                           df['16.1 средства клиентов, не являющихся кредитными организациями'] +
                           df['16.2 средства физических лиц'])
    df['Cash'] = (df['1 Денежные средства и их эквиваленты'] + df['2 Средства в центральных банках'] + 
                  df['3 Средства в кредитных организациях'])
    df['Liabilities'] = df['21 Всего обязательств']
    # Safe funds income (for deposit markups)
    df['Safe_funds'] = df['3 Средства в кредитных организациях']
    df['Safe_funds_income'] = df['1.1 от размещения средств в кредитных организациях']
    df['Safe_rev'] = df['Safe_funds_income'] / df['Safe_funds']

    # For HHI
    df['Sales'] = df['1 Процентные доходы, всего, в том числе:'] + df['13 Комиссионные доходы']
    
    df = df.loc[:, ['regnum', 'dt_year', 'dt_month', 'quarter', 'Name',
                    'Total_assets', 'Net_worth', 'Net_income', 
                    'Loans', 'Admin_expenses', 'Rel_Admin_expenses', 
                    'Securities', 'Cash', 'Liabilities',
                    'NIE', 'NII', 'II_Loans', 'IE_Deposits', 'Leverage', 
                    'Deposits', 'Pure_deposits', 'Sales', 'Safe_funds', 'Safe_funds_income', 'Safe_rev']]
    # Price of Loans
    df['PoL'] = df['II_Loans'] / df['Loans']
    # Cost of Funds
    df['CoF'] = df['IE_Deposits'] / df['Deposits']
    
    
    df = df.replace([np.inf, -np.inf], np.nan)
    return df

In [6]:
def df_var_201903(df):
    """From the central bank data, extract variables for the work. From 2019-03 until 2021-03"""
    # 35 Всего источников собственных средств группы и малых акционеров (участников)
    # 1.2.1, 1.2.2 от кредитов (займов)... (за исключением финансовой аренды)
    # 1.3 от оказания услуг по финансовой аренде (лизингу)
    
    df = df.dropna(subset=['14 Всего активов'])
    
    df['quarter'] = yearmonth_to_quarter(df, 'dt_year', 'dt_month')
    
    df['Name'] = df['Полное или сокращенное фирменное наименование головной кредитной организации']
    df['Total_assets'] = df['14 Всего активов']
    df['Net_worth'] = df['35 Всего источников собственных средств группы и малых акционеров (участников)'] #!
    df['Net_income'] = df['27 Прибыль (убыток) за отчетный период, в том числе:']
    df['Loans'] = (df['4.1 кредиты (займы) и дебиторская задолженность, в том числе:'] +
                   df['5.3 кредиты (займы)'] + df['6.3 кредиты (займы)'])

    df['Admin_expenses'] = df['21 Административные и прочие операционные расходы, в том числе:']
    df['Rel_Admin_expenses'] = df['Admin_expenses'] / df['Total_assets']
    #Total holdings of securities
    df['Securities'] = (df['4.2 долговые ценные бумаги'] + df['5.1 долговые ценные бумаги'] +
                        df['5.2 долевые ценные бумаги'] + df['6.1 долговые ценные бумаги, в том числе:'] +
                        df['6.2 долевые ценные бумаги, в том числе:'])
    # non-interest expenses
    df['NIE'] = (df['14 Комиссионные расходы'] + 
                 df['21 Административные и прочие операционные расходы, в том числе:'])
    # non-interest income
    df['NII'] = (df['12 Доходы в виде дивидендов'] + df['13 Комиссионные доходы'] + df['19 Прочие операционные доходы'])
    # interest income on loans
    df['II_Loans'] = (df['1.2.1 от кредитов (займов), предоставленных кредитным организациям, и дебиторской задолженности (за исключением финансовой аренды)'] +
                      df['1.2.2 от кредитов (займов), предоставленных клиентам, не являющимся кредитными организациями, и дебиторской задолженности (за исключением финансовой аренды)'] +
                      df['1.3 от оказания услуг по финансовой аренде (лизингу)'])
    #interest expenses on deposits
    df['IE_Deposits'] = (df['2.1 по привлеченным средствам кредитных организаций'] +
                         df['2.2 по привлеченным средствам клиентов, не являющихся кредитными организациями'])
    df['Leverage'] = df['Net_worth'] / df['Total_assets']
    df['Deposits'] = (df['15 Финансовые обязательства, оцениваемые по амортизированной стоимости, в том числе:'] + 
                      df['16 Финансовые обязательства, оцениваемые по справедливой стоимости через прибыль или убыток, в том числе:'])
    df['Pure_deposits'] = (df['15.4 средства клиентов, не являющихся кредитными организациями'] +
                           df['15.5 средства физических лиц'] +
                           df['16.1 средства клиентов, не являющихся кредитными организациями'] +
                           df['16.2 средства физических лиц'])
    df['Cash'] = (df['1 Денежные средства и их эквиваленты'] + df['2 Средства в центральных банках'] + 
                  df['3 Средства в кредитных организациях'])
    df['Liabilities'] = df['21 Всего обязательств']
    # Safe funds income (for deposit markups)
    df['Safe_funds'] = df['3 Средства в кредитных организациях']
    df['Safe_funds_income'] = df['1.1 от размещения средств в кредитных организациях']
    df['Safe_rev'] = df['Safe_funds_income'] / df['Safe_funds']
    
    # For HHI
    df['Sales'] = df['1 Процентные доходы, всего, в том числе:'] + df['13 Комиссионные доходы']
    
    df = df.loc[:, ['regnum', 'dt_year', 'dt_month', 'quarter', 'Name',
                    'Total_assets', 'Net_worth', 'Net_income', 
                    'Loans', 'Admin_expenses', 'Rel_Admin_expenses', 
                    'Securities', 'Cash', 'Liabilities',
                    'NIE', 'NII', 'II_Loans', 'IE_Deposits', 'Leverage', 
                    'Deposits', 'Pure_deposits', 'Sales', 'Safe_funds', 'Safe_funds_income', 'Safe_rev']]
    # Price of Loans
    df['PoL'] = df['II_Loans'] / df['Loans']
    # Cost of Funds
    df['CoF'] = df['IE_Deposits'] / df['Deposits']
    
    
    df = df.replace([np.inf, -np.inf], np.nan)
    return df

In [7]:
def df_var_to201812(df):
    """From the central bank data, extract variables for the work. From 2018-09 until 2018-12. And earlier from 2017-09"""
    # 36 Всего источников собственных средств группы и малых акционеров (участников)
    # 15 Всего активов
    # 6 Кредиты (займы)
    # и прочее
    
    df = df.dropna(subset=['15 Всего активов'])
    
    df['quarter'] = yearmonth_to_quarter(df, 'dt_year', 'dt_month')
    
    df['Name'] = df['Полное или сокращенное фирменное наименование головной кредитной организации']
    df['Total_assets'] = df['15 Всего активов']
    df['Net_worth'] = df['36 Всего источников собственных средств группы и малых акционеров (участников)']
    df['Net_income'] = df['26 Прибыль (убыток) за отчетный период']
    df['Loans'] = (df['6 Кредиты (займы) и дебиторская задолженность, в том числе:'])

    df['Admin_expenses'] = df['21 Административные и прочие операционные расходы']
    df['Rel_Admin_expenses'] = df['Admin_expenses'] / df['Total_assets']
    #Total holdings of securities
    df['Securities'] = (df['4 Финансовые активы, оцениваемые по справедливой стоимости через прибыль или убыток, в том числе:'] + 
                        df['5 Финансовые активы, оцениваемые по справедливой стоимости через прибыль или убыток, переданные без прекращения признания'] +
                        df['7 Финансовые активы, имеющиеся в наличии для продажи'])
    # non-interest expenses
    df['NIE'] = (df['14 Комиссионные расходы'] + 
                 df['21 Административные и прочие операционные расходы'])
    # non-interest income
    df['NII'] = (df['12 Доходы в виде дивидендов'] + df['13 Комиссионные доходы'] + df['19 Прочие операционные доходы'])
    # interest income on loans
    df['II_Loans'] = (df['1.1 От размещения средств в кредитных организациях'] +
                      df['1.2 От кредитов (займов), предоставленных клиентам, не являющимся кредитными организациями, и дебиторской задолженности (за исключением финансовой аренды)'] + 
                      df['1.3 От оказания услуг по финансовой аренде (лизингу)'])
    #interest expenses on deposits
    df['IE_Deposits'] = (df['2.1 По привлеченным средствам кредитных организаций'] +
                         df['2.2 По привлеченным средствам клиентов, не являющихся кредитными организациями'])
    df['Leverage'] = df['Net_worth'] / df['Total_assets']
    df['Deposits'] = (df['16 Кредиты, депозиты и прочие средства центральных  банков'] + 
                      df['17 Средства кредитных организаций'] + 
                      df['18 Средства клиентов, не являющихся кредитными организациями'] + 
                      df['19 Финансовые обязательства, оцениваемые по справедливой стоимости через прибыль или убыток, в том числе:'] +
                      df['20 Выпущенные долговые обязательства'])
    df['Pure_deposits'] = (df['18 Средства клиентов, не являющихся кредитными организациями'])
                           #+ df['18.1 Вклады физических лиц']
    df['Cash'] = (df['1 Денежные средства и их эквиваленты'] + df['2 Средства в центральных банках'] + 
                  df['3 Средства в кредитных организациях'])
    df['Liabilities'] = df['25 Всего обязательств']
    # Safe funds income (for deposit markups)
    df['Safe_funds'] = df['3 Средства в кредитных организациях']
    df['Safe_funds_income'] = df['1.1 От размещения средств в кредитных организациях']
    df['Safe_rev'] = df['Safe_funds_income'] / df['Safe_funds']
    
    # For HHI
    df['Sales'] = df['1 Процентные доходы, всего, в том числе:'] + df['13 Комиссионные доходы']
    
    df = df.loc[:, ['regnum', 'dt_year', 'dt_month', 'quarter', 'Name',
                    'Total_assets', 'Net_worth', 'Net_income', 
                    'Loans', 'Admin_expenses', 'Rel_Admin_expenses', 
                    'Securities', 'Cash', 'Liabilities',
                    'NIE', 'NII', 'II_Loans', 'IE_Deposits', 'Leverage', 
                    'Deposits', 'Pure_deposits', 'Sales', 'Safe_funds', 'Safe_funds_income', 'Safe_rev']]
    # Price of Loans
    df['PoL'] = df['II_Loans'] / df['Loans']
    # Cost of Funds
    df['CoF'] = df['IE_Deposits'] / df['Deposits']
    
    
    df = df.replace([np.inf, -np.inf], np.nan)
    return df

Соберём в один DataFrame данные с 2019-03 по 2021-09. Отметим, что в отчётах собраны аккумулированные квартальные отчёты, т.е. доходы/расходы расчитаны с начала года.

In [8]:
i = (2019, '03')
df_temp = pd.read_excel(path_bankdata+r'\bankdata_'+str(i[0])+'_'+i[1]+'.xlsx')
df_temp = df_temp.dropna(subset=['14 Всего активов']) # drop the rows with missing assets
df_temp = df_var_201903(df_temp)
df_temp

,regnum,dt_year,dt_month,quarter,Name,Total_assets,Net_worth,Net_income,Loans,Admin_expenses,...,IE_Deposits,Leverage,Deposits,Pure_deposits,Sales,Safe_funds,Safe_funds_income,Safe_rev,PoL,CoF
0,2306,2019,3,2019q1,Акционерный коммерческий банк «Абсолют Банк» (...,2.609245e+08,1.921198e+07,4801805.0,1.668763e+08,2917780.0,...,3038707.0,0.073630,2.334732e+08,2.138552e+08,2.623039e+07,2291463.0,55451.0,0.024199,0.071026,0.013015
1,2879,2019,3,2019q1,Акционерный Коммерческий банк «АВАНГАРД» — пуб...,1.145952e+08,2.518064e+07,1230729.0,NaN,2084372.0,...,451509.0,0.219736,8.791022e+07,NaN,4.595871e+06,7835611.0,NaN,NaN,NaN,0.005136
10,1810,2019,3,2019q1,«Азиатско-Тихоокеанский Банк» (Акционерное общ...,8.693354e+07,1.391370e+07,3174270.0,5.169270e+07,1747004.0,...,973783.0,0.160050,7.018333e+07,6.847106e+07,9.717640e+06,1172702.0,17356.0,0.014800,0.125596,0.013875
14,2590,2019,3,2019q1,Акционерный коммерческий банк «АК БАРС» (публи...,5.108334e+08,6.759251e+07,1829795.0,3.701018e+08,3553082.0,...,5755965.0,0.132318,4.321711e+08,3.778641e+08,9.835514e+06,6788940.0,0.0,0.000000,0.018606,0.013319
19,2602,2019,3,2019q1,Акционерный Коммерческий Банк «Алмазэргиэнбанк...,2.670309e+07,2.765313e+06,-277326.0,NaN,613647.0,...,300329.0,0.103558,NaN,NaN,1.783210e+06,300460.0,84872.0,0.282474,NaN,NaN
23,1326,2019,3,2019q1,АКЦИОНЕРНОЕ ОБЩЕСТВО «АЛЬФА-БАНК»,3.245893e+09,4.194109e+08,45903567.0,2.092191e+09,23877956.0,...,37505539.0,0.129213,2.757425e+09,2.284439e+09,8.741940e+07,215855786.0,3589220.0,0.016628,0.025259,0.013602
30,436,2019,3,2019q1,ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО «БАНК „САНКТ-ПЕ...,6.482653e+08,7.910669e+07,1159678.0,4.820058e+08,4356238.0,...,6396255.0,0.122028,5.656691e+08,4.061386e+08,1.787760e+07,9669131.0,0.0,0.000000,0.019489,0.011307
33,2312,2019,3,2019q1,Акционерное общество «Банк ДОМ.РФ»,1.929260e+08,1.204904e+07,-4828448.0,9.746225e+07,4023982.0,...,2632375.0,0.062454,1.580634e+08,1.413146e+08,7.808276e+06,4703077.0,0.0,0.000000,0.044138,0.016654
34,3138,2019,3,2019q1,Акционерное Общество «Банк Жилищного Финансиро...,1.220743e+07,2.384784e+06,802751.0,9.257036e+06,168168.0,...,150548.0,0.195355,9.672254e+06,7.802954e+06,6.403020e+05,260984.0,8159.0,0.031262,0.063597,0.015565
35,3255,2019,3,2019q1,Банк ЗЕНИТ (публичное акционерное общество),2.530100e+08,2.798722e+07,615012.0,1.539283e+08,2617181.0,...,2866920.0,0.110617,2.213818e+08,2.004797e+08,1.246412e+07,9257312.0,10768.0,0.001163,0.025812,0.012950


In [9]:
df = pd.DataFrame()
for i in [(2019, '03'), (2019, '06'), (2019, '09'), (2019, '12'), 
          (2020, '03'), (2020, '06'), (2020, '09'), (2020, '12'),
          (2021, '03')]:
    df_temp = pd.read_excel(path_bankdata+r'\bankdata_'+str(i[0])+'_'+i[1]+'.xlsx')
    df_temp = df_temp.dropna(subset=['14 Всего активов'])
    df_temp = df_var_201903(df_temp)
    df_temp = df_temp.dropna()
    df = pd.concat([df, df_temp])
for i in [(2021, '06'), (2021, '09')]:
    df_temp = pd.read_excel(path_bankdata+r'\bankdata_'+str(i[0])+'_'+i[1]+'.xlsx')
    df_temp = df_temp.dropna(subset=['14 Всего активов'])
    df_temp = df_var_202106(df_temp)
    df_temp = df_temp.dropna()
    df = pd.concat([df, df_temp])
#df.loc[df['regnum']==1000]

Рассмотрим два типа датасетов:
<ol>
  <li> Панель из точек на конец годов.</li>
    <ol>
        <li> 2019-2020 (одинаковая отчётность) </li>
        <li> 2017-2018 (одинаковая отчётность) </li>
        <li> 2017-2020 (до Ковида) </li>
    </ol>
  <li> Деаккумулированная панель из квартальных данных.</li>
    <ol>
        <li> 2019-03 по 2021-09 (примерно один формат данных) </li>
        <li> 2017-09 по 2018-12 (примерно один формат данных) </li>
        <li> 2017-09 по 2019-12 (до системных сдвигов, pre-COVID) </li>
        <li> 2017-09 по 2021-09 (максимальная) </li>
    </ol>
    
</ol>

Форма менялась в июне 2021 и в марте 2019, поэтому создаём 3 функции.
Данные отсутствуют за 2021-12,...,2023-03.

# 2. Панель на конец года и деаккумулированная панель

In [10]:
# the results are saved into
path_local = path+r'\var'

## 2.1 Панель на конец года (2017-2020)

Максимально доступная с годовыми данными.

In [11]:
# Панель из годовых отчётов 2017-2020
df = pd.DataFrame()
for i in [(2017, '12'), 
          (2018, '12')]:
    df_temp = pd.read_excel(path_bankdata+r'\bankdata_'+str(i[0])+'_'+i[1]+'.xlsx')
    #df_temp = df_temp.dropna(subset=['15 Всего активов'])
    df_temp = df_var_to201812(df_temp)
    df_temp = df_temp.dropna()
    df = pd.concat([df, df_temp])
for i in [(2019, '12'), 
          (2020, '12')]:
    df_temp = pd.read_excel(path_bankdata+r'\bankdata_'+str(i[0])+'_'+i[1]+'.xlsx')
    #df_temp = df_temp.dropna(subset=['14 Всего активов'])
    df_temp = df_var_201903(df_temp)
    df_temp = df_temp.dropna()
    df = pd.concat([df, df_temp])
df.to_excel(path_local+r'\bankdata_2017-2020_y_var.xlsx')
#df.to_csv(path_local+r'\bankdata_2017-2020_var.csv', encoding='utf-8')
df

,regnum,dt_year,dt_month,quarter,Name,Total_assets,Net_worth,Net_income,Loans,Admin_expenses,...,IE_Deposits,Leverage,Deposits,Pure_deposits,Sales,Safe_funds,Safe_funds_income,Safe_rev,PoL,CoF
0,2306,2017,12,2017q4,Акционерный коммерческий банк «Абсолют Банк» (...,3.165208e+08,23943688.0,-5087365.0,1.696330e+08,-16607696.0,...,18633295.0,0.075646,2.859441e+08,2.201219e+08,34425851.0,5679014.0,307128.0,0.054081,0.136013,0.065164
10,1810,2017,12,2017q4,«Азиатско-Тихоокеанский Банк» (Акционерное общ...,1.051164e+08,13697749.0,-536147.0,6.181845e+07,17183619.0,...,6420222.0,0.130310,8.847114e+07,8.167237e+07,17126278.0,2122262.0,128753.0,0.060668,0.195958,0.072569
14,2590,2017,12,2017q4,Акционерный коммерческий банк «АК БАРС» (публи...,3.961865e+08,63900838.0,900927.0,2.320637e+08,25840513.0,...,21847637.0,0.161290,3.173214e+08,2.871494e+08,37657722.0,5806602.0,2498166.0,0.430229,0.115502,0.068850
23,1326,2017,12,2017q4,АКЦИОНЕРНОЕ ОБЩЕСТВО «АЛЬФА-БАНК»,2.635086e+09,280754748.0,51062318.0,1.565370e+09,98182438.0,...,86588677.0,0.106545,2.296712e+09,2.031833e+09,282402794.0,239171171.0,18565305.0,0.077624,0.117387,0.037701
30,436,2017,12,2017q4,ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО «БАНК „САНКТ-ПЕ...,5.975045e+08,56311373.0,4061287.0,4.068525e+08,13331756.0,...,26717858.0,0.094244,5.218685e+08,3.626863e+08,53080099.0,12352526.0,4028424.0,0.326121,0.094803,0.051197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,316,2020,12,2020q4,Общество с ограниченной ответственностью «Хоум...,2.809852e+08,65697720.0,4516202.0,2.294914e+08,25935173.0,...,13050463.0,0.233812,2.092369e+08,1.772766e+08,86440204.0,1207485.0,0.0,0.000000,0.231741,0.062372
338,485,2020,12,2020q4,АКЦИОНЕРНЫЙ КОММЕРЧЕСКИЙ БАНК «ЧЕЛИНДБАНК» (пу...,5.845194e+07,11915788.0,1067164.0,2.837713e+07,6709147.0,...,1840602.0,0.203856,4.395526e+07,4.310285e+07,9928400.0,482815.0,54858.0,0.113621,0.256399,0.041874
339,493,2020,12,2020q4,АКЦИОНЕРНЫЙ ЧЕЛЯБИНСКИЙ ИНВЕСТИЦИОННЫЙ БАНК «Ч...,5.870302e+07,10418863.0,1251349.0,2.792852e+07,9441039.0,...,1494980.0,0.177484,4.728040e+07,4.402206e+07,12482430.0,1453019.0,0.0,0.000000,0.096637,0.031619
344,2998,2020,12,2020q4,Акционерное общество «Экспобанк»,1.007195e+08,21403346.0,3385891.0,7.424404e+07,4086631.0,...,2916334.0,0.212504,7.762820e+07,7.524270e+07,22622028.0,2269704.0,298887.0,0.131685,0.107824,0.037568


## 2.2 Деаккумулированная панель из квартальных данных 

In [12]:
def deaccumulate(df):
    """
    Given a DataFrame with quaterly accumulated data, the function returns DataFrame with non-accumulated quaterly data.
    If no previous quaterly data is missing (and not the first quater of the year) then the row would be deleted
    """
    
    #Direct updating: 'Net_income', 'Admin_expenses', 'NIE', 'NII', 'II_Loans', 'IE_Deposits', 
    #Indirect updating: 'Rel_Admin_expenses', 'PoL', 'CoF'

    # if month = '03': nothing
    # if month = '06': -'03' of the same year
    # if month = '09': -'06' of the same year
    # if month = '12': -'09' of the same year
    # if previous month is missing then put flag to delete the row at the end (this is for 2017q3), as deaccumulation is impossible
    # update all indirectly updated fields
    
    df['flag'] = 0
    years = np.sort(df['dt_year'].unique())[::-1]
    for regnum in df['regnum'].unique():
        for year in years:
            for month in [12, 9, 6]:
                condition_0 = (df['regnum'] == regnum) & (df['dt_year'] == year) & (df['dt_month'] == month)
                condition_1 = (df['regnum'] == regnum) & (df['dt_year'] == year) & (df['dt_month'] == month-3)
                if ((len(df.loc[condition_0]['Net_income']) > 0) & (len(df.loc[condition_1]['Net_income']) > 0)):
                    #i = df.loc[condition_0].index[0] #index of the row
                    #print('yes')
                    df.loc[condition_0, 'Net_income'] = df.loc[condition_0, 'Net_income'] - df.loc[condition_1, 'Net_income']
                    df.loc[condition_0, 'Admin_expenses'] = df.loc[condition_0, 'Admin_expenses'] - df.loc[condition_1, 'Admin_expenses']
                    df.loc[condition_0, 'NIE'] = df.loc[condition_0, 'NIE'] - df.loc[condition_1, 'NIE']
                    df.loc[condition_0, 'NII'] = df.loc[condition_0, 'NII'] - df.loc[condition_1, 'NII']
                    df.loc[condition_0, 'II_Loans'] = df.loc[condition_0, 'II_Loans'] - df.loc[condition_1, 'II_Loans']
                    df.loc[condition_0, 'IE_Deposits'] = df.loc[condition_0, 'IE_Deposits'] - df.loc[condition_1, 'IE_Deposits']
                    df.loc[condition_0, 'Safe_funds_income'] = df.loc[condition_0, 'Safe_funds_income'] - df.loc[condition_1, 'Safe_funds_income']
                    df.loc[condition_0, 'Sales'] = df.loc[condition_0, 'Sales'] - df.loc[condition_1, 'Sales']
                elif ((len(df.loc[condition_0]['Net_income']) > 0) & (len(df.loc[condition_1]['Net_income']) == 0)):
                    #i = df.loc[condition_0].index[0] #index of the row
                    #print('no')
                    df.loc[condition_0, 'flag'] = 1
    df['Rel_Admin_expenses'] = df['Admin_expenses'] / df['Total_assets']
    df['PoL'] = df['II_Loans'] / df['Loans'] # Price of Loans
    df['CoF'] = df['IE_Deposits'] / df['Deposits'] # Cost of Funds
    df['Safe_rev'] = df['Safe_funds_income'] / df['Safe_funds']
    df = df.loc[df['flag'] == 0]
    
    df = df.replace([np.inf, -np.inf], np.nan)
    return df

### Панель с 2017-09 по 2021-09 (максимальная)

In [13]:
#Деаккумулированная панель из квартальных данных с 2017-09 по 2021-09.
df = pd.DataFrame()
for i in [(2017, '09'), (2017, '12'), 
          (2018, '03'), (2018, '06'), (2018, '09'), (2018, '12')]:
    df_temp = pd.read_excel(path_bankdata+r'\bankdata_'+str(i[0])+'_'+i[1]+'.xlsx')
    #df_temp = df_temp.dropna(subset=['15 Всего активов'])
    df_temp = df_var_to201812(df_temp)
    df_temp = df_temp.dropna()
    df = pd.concat([df, df_temp])
for i in [(2019, '03'), (2019, '06'), (2019, '09'), (2019, '12'), 
          (2020, '03'), (2020, '06'), (2020, '09'), (2020, '12'),
          (2021, '03')]:
    df_temp = pd.read_excel(path_bankdata+r'\bankdata_'+str(i[0])+'_'+i[1]+'.xlsx')
    #df_temp = df_temp.dropna(subset=['14 Всего активов'])
    df_temp = df_var_201903(df_temp)
    df_temp = df_temp.dropna()
    df = pd.concat([df, df_temp])
for i in [(2021, '06'), (2021, '09')]:
    df_temp = pd.read_excel(path_bankdata+r'\bankdata_'+str(i[0])+'_'+i[1]+'.xlsx')
    #df_temp = df_temp.dropna(subset=['14 Всего активов'])
    df_temp = df_var_202106(df_temp)
    df_temp = df_temp.dropna()
    df = pd.concat([df, df_temp])
df.to_excel(path_local+r'\bankdata_201903-202109_nodeacc.xlsx')

df = deaccumulate(df)
df.to_excel(path_local+r'\bankdata_201709-202109_q_var.xlsx')
#df.to_csv(path_local+r'\bankdata_201709-202109_var.csv', encoding='utf-8')
df

,regnum,dt_year,dt_month,quarter,Name,Total_assets,Net_worth,Net_income,Loans,Admin_expenses,...,Leverage,Deposits,Pure_deposits,Sales,Safe_funds,Safe_funds_income,Safe_rev,PoL,CoF,flag
0,2306,2017,12,2017q4,Акционерный коммерческий банк «Абсолют Банк» (...,3.165208e+08,23943688.0,-1061276.0,1.696330e+08,-5606076.0,...,0.075646,2.859441e+08,2.201219e+08,9628099.0,5679014.0,81965.0,0.014433,0.039690,0.015513,0
14,2590,2017,12,2017q4,Акционерный коммерческий банк «АК БАРС» (публи...,3.961865e+08,63900838.0,-1156428.0,2.320637e+08,5854174.0,...,0.161290,3.173214e+08,2.871494e+08,9357237.0,5806602.0,714093.0,0.122979,0.027174,0.017280,0
23,1326,2017,12,2017q4,АКЦИОНЕРНОЕ ОБЩЕСТВО «АЛЬФА-БАНК»,2.635086e+09,280754748.0,41942751.0,1.565370e+09,25735107.0,...,0.106545,2.296712e+09,2.031833e+09,77416155.0,239171171.0,4060896.0,0.016979,0.031130,0.009093,0
30,436,2017,12,2017q4,ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО «БАНК „САНКТ-ПЕ...,5.975045e+08,56311373.0,654477.0,4.068525e+08,3887195.0,...,0.094244,5.218685e+08,3.626863e+08,13435793.0,12352526.0,1112921.0,0.090097,0.023613,0.011137,0
33,2312,2017,12,2017q4,Акционерное общество «Банк ДОМ.РФ»,3.580154e+08,41862030.0,-2573067.0,1.950314e+08,10924468.0,...,0.116928,2.971968e+08,2.287697e+08,10448377.0,9452868.0,80733.0,0.008541,0.040771,0.017990,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,316,2021,9,2021q3,Общество с ограниченной ответственностью «Хоум...,3.358071e+08,71461951.0,2886458.0,2.796779e+08,6520303.0,...,0.212807,2.576047e+08,2.110302e+08,22040194.0,2560474.0,0.0,0.000000,0.047520,0.010131,0
338,485,2021,9,2021q3,АКЦИОНЕРНЫЙ КОММЕРЧЕСКИЙ БАНК «ЧЕЛИНДБАНК» (пу...,6.084572e+07,12719019.0,316371.0,3.144199e+07,707604.0,...,0.209037,4.556829e+07,4.312986e+07,1639210.0,479860.0,12725.0,0.026518,0.025250,0.008746,0
339,493,2021,9,2021q3,АКЦИОНЕРНЫЙ ЧЕЛЯБИНСКИЙ ИНВЕСТИЦИОННЫЙ БАНК «Ч...,6.165321e+07,11473243.0,421155.0,3.042697e+07,2899317.0,...,0.186093,4.919869e+07,4.673456e+07,3656805.0,1655546.0,0.0,0.000000,0.024386,0.006256,0
344,2998,2021,9,2021q3,Акционерное общество «Экспобанк»,1.540950e+08,24112637.0,1870489.0,1.204652e+08,3953194.0,...,0.156479,1.280142e+08,1.078273e+08,6046001.0,1593693.0,132728.0,0.083283,0.024992,0.007641,0


Next steps:
* Prepare cross-section datasets and estimate credit markups there
* Prepare full (longest possible) ~~yearly and~~ quaterly panel data and estimate credit markups on it - ```done!```
* Try before 2020: 2017, 2018, 2019 yearly data - ```done!```
* Try before 2020: monthly data - ```done!```

# 3 Кросс-секции as is (no deaccumulation) с 2017-09 по 2021-09

In [14]:
path_local = path+r'\cross-section'

In [15]:
for i in [(2017, '09'), (2017, '12'), 
          (2018, '03'), (2018, '06'), (2018, '09'), (2018, '12')]:
    df_temp = pd.read_excel(path_bankdata+r'\bankdata_'+str(i[0])+'_'+i[1]+'.xlsx')
    #df_temp = df_temp.dropna(subset=['15 Всего активов'])
    df_temp = df_var_to201812(df_temp)
    df_temp = df_temp.dropna()
    df_temp.to_csv(path_local+r'\bankdata_'+str(i[0])+'_'+i[1]+'_var.csv', encoding='utf-8')
    df_temp.to_excel(path+r'\cross-section\bankdata_'+str(i[0])+'_'+i[1]+'_var.xlsx')
for i in [(2019, '03'), (2019, '06'), (2019, '09'), (2019, '12'), 
          (2020, '03'), (2020, '06'), (2020, '09'), (2020, '12'),
          (2021, '03')]:
    df_temp = pd.read_excel(path_bankdata+r'\bankdata_'+str(i[0])+'_'+i[1]+'.xlsx')
    #df_temp = df_temp.dropna(subset=['14 Всего активов'])
    df_temp = df_var_201903(df_temp)
    df_temp = df_temp.dropna()
    df_temp.to_csv(path_local+r'\bankdata_'+str(i[0])+'_'+i[1]+'_var.csv', encoding='utf-8')
    df_temp.to_excel(path_local+r'\bankdata_'+str(i[0])+'_'+i[1]+'_var.xlsx')
for i in [(2021, '06'), (2021, '09')]:
    df_temp = pd.read_excel(path_bankdata+r'\bankdata_'+str(i[0])+'_'+i[1]+'.xlsx')
    #df_temp = df_temp.dropna(subset=['14 Всего активов'])
    df_temp = df_var_202106(df_temp)
    df_temp = df_temp.dropna()
    df_temp.to_csv(path_local+r'\bankdata_'+str(i[0])+'_'+i[1]+'_var.csv', encoding='utf-8')
    df_temp.to_excel(path_local+r'\bankdata_'+str(i[0])+'_'+i[1]+'_var.xlsx')

# 4 Поправка на квартальный индекс потребительских цен (CPI)

In [16]:
# quarterly consumer price index (CPI) taken from Rosstat
df_CPI = pd.read_excel(path+r'\external_data\CPI_quarterly.xlsx', sheet_name='Q2Q', index_col='quarter')
df_CPI

,End Q to end Q,Q to Q,Q to last year Q
quarter,,,
2005q1,105.27,105.09,113.09
2005q2,102.58,103.20,113.76
2005q3,100.57,101.15,112.67
2005q4,102.12,101.44,111.28
2006q1,104.98,104.67,110.84
...,...,...,...
2022q4,101.33,100.55,112.18
2023q1,101.67,101.92,108.62
2023q2,101.06,101.12,102.69


In [17]:
def CPI_accumulate(df, df_CPI, col='Q to Q'):
    """ Function returns a CPI with column 'Q2Q_acc' that contains multipliers for df
    df_CPI.index must be quarter, df must contain column 'quarter'
    
    This is to correct the data compared to the initial time period in the given dataframe df
    """
    df_CPI_curr = df_CPI.loc[(df_CPI.index>=min(df['quarter'])) & (df_CPI.index<=max(df['quarter']))]
    
    cpi = 1
    cpi_acc = []
    for q in df_CPI_curr.index:
        if q==min(df_CPI_curr.index):
            cpi = 1
        else:
            #cpi *= np.array(df_CPI_curr.loc[df_CPI_curr.index == q]['Q to Q'])[0]/100
            cpi *= df_CPI_curr.at[q, col]/100
        cpi_acc.append(cpi)
    df_CPI_curr[col+' Acc'] = cpi_acc
    return df_CPI_curr

In [18]:
def to_real_values(df, df_CPI, col='Q to Q'):
    """Function returns data with nominal values corrected for CPI"""
    
    df_CPI_curr = CPI_accumulate(df, df_CPI, col)
    
    cols = ['Total_assets', 'Net_worth', 'Net_income', 'Loans',
       'Admin_expenses', 'Securities', 'Cash', 'Liabilities', 'NIE', 'NII',
       'II_Loans', 'IE_Deposits', 'Deposits', 'Pure_deposits', 'Sales', 'Safe_funds', 'Safe_funds_income']  
    
    for c in cols:
        for q in df_CPI_curr.index:
            for ind in df.loc[df['quarter']==q].index:
                df.at[ind, c] /= df_CPI_curr[col+' Acc'][q]
    return df

In [19]:
path_from = path + r'\var'
path_to = path + r'\var_real'

Панель с 2017-09 по 2021-09 (максимальная)

In [20]:
df = pd.read_excel(path+r'\var\bankdata_201709-202109_q_var.xlsx')
df = to_real_values(df, df_CPI)
df.to_excel(path+r'\var\bankdata_201709-202109_q_var_real.xlsx')
#df.to_csv(path+r'\var_real\bankdata_201709-202109_var_real.csv', encoding='utf-8')

## 5 Поправка на годовой индекс потребительских цен (CPI)

Prepare the reference dataframe with yearly CPI

In [21]:
def dates_to_years(dates):
    """An array of dates to format YYYY (2003-09-01 to 2003)"""
    years = []
    for date in dates:
        years.append(date.year)
    return years

In [22]:
def date_to_quarter(date):
    """Date to format YYYYqN (2003-09-01 to 2003q3)"""
    month_to_quarter = {1: 'q1', 2: 'q1', 3: 'q1',
                        4: 'q2', 5: 'q2', 6: 'q2',
                        7: 'q3', 8: 'q3', 9: 'q3',
                        10: 'q4', 11: 'q4', 12: 'q4'}
    quarter = str(date.year) + month_to_quarter[date.month]
    return quarter

In [23]:
# read the Rosstat data on quarterly CPI (Consumer price index)
df_CPI = pd.read_excel(path+r'\external_data\CPI_quarterly.xlsx', sheet_name='Q2Q', index_col='quarter')
#df_CPI

In [24]:
df_CPI['date'] = quarter_to_date(df_CPI.index)
df_CPI['year'] = dates_to_years(df_CPI['date'])
df_CPI

,End Q to end Q,Q to Q,Q to last year Q,date,year
quarter,,,,,
2005q1,105.27,105.09,113.09,2005-01-01,2005
2005q2,102.58,103.20,113.76,2005-04-01,2005
2005q3,100.57,101.15,112.67,2005-07-01,2005
2005q4,102.12,101.44,111.28,2005-10-01,2005
2006q1,104.98,104.67,110.84,2006-01-01,2006
...,...,...,...,...,...
2022q4,101.33,100.55,112.18,2022-10-01,2022
2023q1,101.67,101.92,108.62,2023-01-01,2023
2023q2,101.06,101.12,102.69,2023-04-01,2023


In [25]:
def yearly_cpi_accumulation(df, column):
    """Given data on quarter-to-quarter basis in a column, aggregate it into yearly data as geometric mean,
    i.e. use quarter to the same quarter of past year index for 4 quarters and get the geometric mean as an
    average inflation during the year
    
    df -- dataframe with CPI indices
    column -- the column name for CPI index
    """
    res = pd.DataFrame()
    for year in df['year'].unique():
        ind = 1
        count = 0
        for date in df.loc[df['year']==year]['date']:
            index = date_to_quarter(date)
            ind *= df.loc[df.index == index][column][index]
            count += 1
        ind = ind**(1/count)
        df_temp = pd.DataFrame({'year': [year], column: [ind], 'obs': [count]})
        res = pd.concat([res, df_temp], ignore_index = True)
        res.reset_index()
    return res

In [26]:
# dataset with yearly CPI index
df_CPI_yearly = yearly_cpi_accumulation(df_CPI, 'Q to last year Q')
df_CPI_yearly = df_CPI_yearly.groupby(['year']).mean()
df_CPI_yearly

,Q to last year Q,obs
year,,
2005,112.696337,4.0
2006,109.685389,4.0
2007,108.980338,4.0
2008,114.099294,4.0
2009,111.687674,4.0
2010,106.846404,4.0
2011,108.451093,4.0
2012,105.052770,4.0
2013,106.754304,4.0


In [27]:
def CPI_accumulate_yearly(df, df_CPI, col='Q to last year Q'):
    """ Function returns a CPI with column 'CPI_acc' that contains multipliers for df
    df_CPI.index must be year, df must contain column 'dt_year'
    
    This is to correct the data compared to the initial time period in the given dataframe df
    """
    df_CPI_curr = df_CPI.loc[(df_CPI.index>=min(df['dt_year'])) & (df_CPI.index<=max(df['dt_year']))]
    
    cpi = 1
    cpi_acc = []
    for year in df_CPI_curr.index:
        if year==min(df_CPI_curr.index):
            cpi = 1
        else:
            #cpi *= np.array(df_CPI_curr.loc[df_CPI_curr.index == q]['Q to Q'])[0]/100
            cpi *= df_CPI_curr.at[year, col]/100
        cpi_acc.append(cpi)
    df_CPI_curr[col+' Acc'] = cpi_acc
    return df_CPI_curr

In [28]:
def to_real_values_yearly(df, df_CPI_yearly, col='Q to last year Q'):
    """Function returns data with nominal values corrected for CPI, yearly data"""
    
    df_CPI_curr = CPI_accumulate_yearly(df, df_CPI_yearly, col)
    
    cols = ['Total_assets', 'Net_worth', 'Net_income', 'Loans',
       'Admin_expenses', 'Securities', 'Cash', 'Liabilities', 'NIE', 'NII',
       'II_Loans', 'IE_Deposits', 'Deposits', 'Pure_deposits', 'Sales', 'Safe_funds', 'Safe_funds_income']    
    
    for c in cols:
        for q in df_CPI_curr.index:
            for ind in df.loc[df['quarter']==q].index:
                df.at[ind, c] /= df_CPI_curr[col+' Acc'][q]
    return df

Панель 2017-2020 (максимальные годовые данные)

In [29]:
df = pd.read_excel(path+r'\var\bankdata_2017-2020_y_var.xlsx')

In [30]:
CPI_accumulate_yearly(df, df_CPI_yearly)

,Q to last year Q,obs,Q to last year Q Acc
year,,,
2017,103.686994,4.0,1.000000
2018,102.862989,4.0,1.028630
2019,104.467794,4.0,1.074587
2020,103.374910,4.0,1.110853


In [31]:
df = to_real_values_yearly(df, df_CPI_yearly)
df.to_excel(path+r'\var\bankdata_2017-2020_y_var_real.xlsx')
#df.to_csv(path+r'\var\bankdata_2017-2020_var_real.csv', encoding='utf-8')